In [1]:
import sys
import json
import math
from collections import Counter
import re
from nltk.corpus import stopwords
import string
from collections import defaultdict
import operator

#Variables para StopWords
punctuation = list(string.punctuation)
stop = stopwords.words('spanish') + punctuation + ['rt', 'via']
#Variables para PMI
semantic_orientation = {}
p_t = {}
p_t_com = defaultdict(lambda : defaultdict(int))
com = defaultdict(lambda : defaultdict(int))
pmi = defaultdict(lambda : defaultdict(int))


positive_vocab = [':-)', ':)', ':o)', ':]', ':3', ':c)', ':D', 'C:',
                  'bien', 'bueno', 'YES', 'World', 'educativas', 'deber',
                  'vida', 'religioso',
    'DELINKUENTE', 'SI', '#UNAC', 'UNAC', 'asumir', 'ASUMIR']
negative_vocab = [':-(', ':(', ':c', ':[', 'D8', 'D;', 'D=', 'DX', 'v.v',
    'malo', 'mal','NO', 'error']
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
  
n_docs = 0
fname = 'tweet_Congreso.json'
with open(fname, 'r') as f:
    count_stop_single = Counter()
    for line in f:
        n_docs+=1
        tweet = json.loads(line)
        print(tweet["text"])
        print("="*150)
        terms_only = [term for term in preprocess(tweet["text"]) 
                      if term not in stop and not term.startswith(("@", "http")) and len(term) > 2]
        count_stop_single.update(terms_only)
            
# Build co-occurrence matrix
for i in range(len(terms_only)-1):
    for j in range(i+1, len(terms_only)):
        w1, w2 = sorted([terms_only[i], terms_only[j]])
        if w1 != w2:
            com[w1][w2] += 1
            com_max = []

for t1 in com:
    t1_max_terms = sorted(com[t1].items(), key=operator.itemgetter(1), reverse=True)[:5]
    for t2, t2_count in t1_max_terms:
        com_max.append(((t1, t2), t2_count))
        terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)


for term, n in count_stop_single.items():
    p_t[term] = n / n_docs
    for t2 in com[term]:
        p_t_com[term][t2] = com[term][t2] / n_docs

pmi = defaultdict(lambda : defaultdict(int))
for t1 in p_t:
    for t2 in com[t1]:
        denom = p_t[t1] * p_t[t2]
        pmi[t1][t2] = math.log2(p_t_com[t1][t2] / denom)


for term, n in p_t.items():
    positive_assoc = sum(pmi[term][tx] for tx in positive_vocab)
    negative_assoc = sum(pmi[term][tx] for tx in negative_vocab)
    semantic_orientation[term] = positive_assoc - negative_assoc
    semantic_sorted = sorted(semantic_orientation.items(), 
                         key=operator.itemgetter(1), 
                         reverse=True)
    top_pos = semantic_sorted[:10]
    top_neg = semantic_sorted[-10:]
    
print("#######", "Base de palabras significativas", "##########")
print(top_pos, "top_pos")
print(('=')*100)
print(top_neg)
print("#######", "ANALYSIS", "##########")
print("UNAC: %f" % semantic_orientation['#Congreso'])

          




RT @ideeleradio: [#VIDEO] Vicepresidenta del #TC: No tengo miedo a la decisión que tome el #Congreso tras resolución en caso Frontón https:…
[#VIDEO] Vicepresidenta del #TC: No tengo miedo a la decisión que tome el #Congreso tras resolución en caso Frontón… https://t.co/xns5YZLsGP
RT @_diodres: @suriel El #SuperintendenteDeMigraciones debería ser destituido x el #Congreso x cometer abuso contra #KorinaRivadeneira hay…
RT @_diodres: @suriel El #SuperintendenteDeMigraciones debería ser destituido x el #Congreso x cometer abuso contra #KorinaRivadeneira hay…
RT @_diodres: @suriel El #SuperintendenteDeMigraciones debería ser destituido x el #Congreso x cometer abuso contra #KorinaRivadeneira hay…
@suriel El #SuperintendenteDeMigraciones debería ser destituido x el #Congreso x cometer abuso contra… https://t.co/d2U50O3Edy
#Congreso 📚📖 
#GYQ_ITB😉 
#Pasito_a_Pasito👣 
#Falta_poco💪
Becerril le dice a Kenji que tome una mototaxi y vaya al #Congreso ... - Diario Perú21 https://t.co/aFj4U6Ysgy htt